<a href="https://colab.research.google.com/github/lannd3217/Interview_RAG/blob/main/InterviewRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/lannd3217/Interview_RAG.git

fatal: destination path 'Interview_RAG' already exists and is not an empty directory.


In [2]:
%cd "Interview_RAG"

/content/Interview_RAG


In [7]:
!pip install -U langchain langchain-community langchain-openai
!pip install -U langchain langchain-community langchain-text-splitters
!pip install -U pymupdf langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 69.3 MB/s eta 0:00:00


In [4]:
!pip install -q sentence-transformers faiss-cpu transformers


In [8]:
import os
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader

def load_pdfs_from_folder(folder_path="./docs"):
    """
    Loads all PDF documents from a specified directory.
    """
    print(f"Loading PDFs from: {folder_path}...")

    # Initialize the DirectoryLoader to specifically target PDF files
    # Using PyMuPDFLoader for its speed and robust text extraction
    loader = DirectoryLoader(
        folder_path,
        glob="*.pdf",
        loader_cls=PyMuPDFLoader,
        show_progress=True,
        silent_errors=True  # Skips files that might be corrupted
    )

    # Load the documents into a list of LangChain Document objects
    docs = loader.load()

    print(f"Successfully loaded {len(docs)} pages from your PDFs.")
    return docs

# Example usage
pdf_documents = load_pdfs_from_folder("./docs")

Loading PDFs from: ./docs...


100%|██████████| 9/9 [00:11<00:00,  1.30s/it]

Successfully loaded 1429 pages from your PDFs.


In [16]:
# Print the first 500 characters of the very first page loaded
len(pdf_documents)
# print(pdf_documents[8].page_content[:500])

1429

In [17]:
search_term = "A/B testing"
for doc in pdf_documents:
    if search_term.lower() in doc.page_content.lower():
        print(f"Found '{search_term}' in {doc.metadata['source']} on page {doc.metadata['page']}")

Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 0
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 1
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 2
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 3
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 4
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 5
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 6
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 7
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 8
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 10
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 12
Found 'A/B testing' in docs/A:B Testing- A SystematicLiterature.pdf on page 13
Found 'A/B testing' in docs/A:B Testing- A SystematicLiteratu

In [19]:
import re

def clean_text(text):
    # 1. Remove weird "cid" characters (often found in PDFs with special fonts)
    text = re.sub(r'\(cid:\d+\)', '', text)

    # 2. Replace multiple newlines with a single one to keep structure but save space
    text = re.sub(r'\n\s*\n', '\n', text)

    # 3. Replace multiple spaces with a single space
    text = re.sub(r' +', ' ', text)

    # 4. Remove purely numerical lines (often page numbers or footers)
    text = re.sub(r'^\d+$\n', '', text, flags=re.MULTILINE)

    return text.strip()

# --- Apply cleaning to your existing documents ---
cleaned_samples = []

for doc in pdf_documents:
    # Extract raw content
    raw_content = doc.page_content

    # Clean it
    cleaned_content = clean_text(raw_content)

    # Store a preview (First 200 chars) for printing
    source_name = doc.metadata.get('source', 'Unknown File')
    cleaned_samples.append((source_name, cleaned_content))

# --- Print the Example Output ---
print(f"✨ Cleaned {len(pdf_documents)} pages.\n")
for filename, text in cleaned_samples[:3]: # Preview first 3 pages
    print(f"Source: {filename}")
    print(f"--- Cleaned Text Preview ---\n{text[:350]}...")
    print("-" * 30)

✨ Cleaned 1429 pages.

Source: docs/Meta_ML_initial_interview_prep.pdf
--- Cleaned Text Preview ---
1 
Machine Learning 
Initial 
Interview Guide 
Welcome to your preparation guide for your Machine Learning (ML) 
initial interview at Meta! Use the sidebar to quickly jump to the 
section you are looking for. Our ML engineering leaders and 
recruiters put together this guide, so you know what to expect and 
how to prepare. We recognize that intervi...
------------------------------
Source: docs/Meta_ML_initial_interview_prep.pdf
--- Cleaned Text Preview ---
2 
Coding (35 minutes) 
You'll solve two coding problems focused on CS fundamentals like algorithms, 
data structures, recursions, and binary trees. If your tech screen is by phone, the 
engineer will send you a collaborative editor (such as coderpad.io). If your tech 
screen is in person, you'll use a whiteboard. 
Answering Your Questions (5 minut...
------------------------------
Source: docs/Meta_ML_initial_interview_prep.pdf
--- C